画像をクラスタリングして似ている画像や完全に同じ画像が無いかを探索します。

画像間の距離は ImageHash で計算したハッシュ値の差分を用います。

クラスタリングは逐次クラスタリングの手法を用います。train/test 全ての画像に対して以下の手続きを行い、所属するクラスターを求めます。

1. 最初、クラスターは1つも存在しません。
2. 1枚目の画像はクラスター1に所属するとします。
2. $K(\gt1)$番目の画像について、既存のクラスター全てとの距離を計算します。クラスターとの距離とは「当該クラスターに属する画像の中で、最も距離が近いものとの距離」と定義します。
3, 最も距離が近いクラスターが$i$番目のクラスターだったとします。
  - クラスター$i$との距離が一定の閾値（今回は10.0とします）未満であれば$K$番目の画像はクラスター$i$に属するとします。
  - クラスター$i$との距離が閾値を超えた場合、$K$番目は既存クラスターのどれにも属さないので、$K$番目の画像だけが所属する新たなクラスターを生成します。


## モジュールの import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob
import os

try:
    import imagehash
except ImportError:
    !pip install Imagehash --quiet
    import imagehash
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from PIL import Image
import seaborn as sns
from tqdm.notebook import tqdm

## データの読み込み

In [ ]:
DATA_DIR = '/content/drive/MyDrive/nishika/'  # コンペのデータを置いているフォルダ
assert os.path.isdir(DATA_DIR)

# train/test の画像フォルダも存在しなければNG
assert os.path.isdir(os.path.join(DATA_DIR, 'train'))
assert os.path.isdir(os.path.join(DATA_DIR, 'test'))

In [ ]:
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

# ファイル名には重複が無い
assert not train['odai_photo_file_name'].duplicated().any()
assert not test['odai_photo_file_name'].duplicated().any()

# train/test の両方に登場するファイル名も存在しない
assert not train['odai_photo_file_name'].isin(test['odai_photo_file_name']).any()

In [ ]:
train

,id,odai_photo_file_name,text,is_laugh
0,ge5kssftl,9fkys1gb2r.jpg,君しょっちゅうソレ自慢するけど、ツムジ２個ってそんなに嬉しいのかい？,0
1,r7sm6tvkj,c6ag0m1lak.jpg,これでバレない？授業中寝てもバレない？,0
2,yp5aze0bh,whtn6gb9ww.jpg,「あなたも感じる？」\n『ああ…、感じてる…』\n「後ろに幽霊いるよね…」\n『女のな…』,0
3,ujaixzo56,6yk5cwmrsy.jpg,大塚愛聞いてたらお腹減った…さく、らんぼと牛タン食べたい…,0
4,7vkeveptl,0i9gsa2jsm.jpg,熊だと思ったら嫁だった,0
...,...,...,...,...
24957,xa2nruec1,5ctq9ohpge.jpg,え、いいんすか？マジっすか？あざーっす,0
24958,dl8r1idfv,dcj9pepjwf.jpg,しかし回り込まれてしまった！,1
24959,kabzw7bxm,ks04y4iy7i.jpg,緑でもない中を走り抜けてく♪真っ赤でもないポルシェでもない♪,1
24960,4blagy0gf,cgfkktchbz.jpg,後ろのサングラスの友達を撮ろうとしたが、思いっきり割り込んできた。,0


In [ ]:
test

,id,odai_photo_file_name,text
0,rfdjcfsqq,nc1kez326b.jpg,僕のママ、キャラ弁のゆでたまごに８時間かかったんだ
1,tsgqmfpef,49xt2fmjw0.jpg,かわいいが作れた！
2,owjcthkz2,9dtscjmyfh.jpg,来世の志茂田景樹
3,rvgaocjyy,osa3n56tiv.jpg,ちょ、あの、オカン、これ水風呂やねんけど、なんの冗談??
4,uxtwu5i69,yb1yqs4pvb.jpg,「今日は皆さんにザリガニと消防車の違いを知ってもらいたいと思います」『どっちも同じだろ。両方...
...,...,...,...
5995,vx1lpzark,0bgwr5po4l.jpg,フォントの幸せってなんなんでしょうね
5996,y9sugbhm8,3wgkjwrq11.jpg,隣に出来た店のせいで自分の店の売り上げが落ちたので無銭飲食しようと乗り込んだらちょうど1万人...
5997,dsd1yixzk,rny98dohwa.jpg,大クラッシュ後、マンガのような形でまさかの生還
5998,vmyopn0mu,rlrze2yhes.jpg,天井にバレーボールがはさまってる


## クラスタリング

In [ ]:
def gen_path_and_text():
    # 画像のファイル名と格納フォルダを順番に返す
    for is_train in (True, False):
        if is_train:
            for filename in train['odai_photo_file_name'].tolist():
                yield filename, os.path.join(DATA_DIR, 'train')
        else:
            for filename in test['odai_photo_file_name'].tolist():
                yield filename, os.path.join(DATA_DIR, 'test')

In [ ]:
%%time

THRESHOLD = 10.  # ハッシュの差分がこれ未満でないと同じクラスターにはなれない
clusters = []  # list of list[filename]
hash_values = {}  # filename: image hash
n = train.shape[0] + test.shape[0]

for i, (filename, directory) in enumerate(gen_path_and_text()):

    # 画像のハッシュ計算処理は重いので結果は cache しておく
    image = Image.open(os.path.join(directory, filename))
    hash_value = imagehash.phash(image)
    hash_values[filename] = hash_value

    # 画像が所属するクラスターを決める
    if not clusters:
        clusters.append([filename])
    else:
        # 最も近くのクラスターを特定する
        distance_from_nearest_cluster = nearest_cluster_idx = None
        for cluster_idx, image_filenames_in_cluster in enumerate(clusters):
            distance_from_cluster = min([hash_value - hash_values[f] for f in image_filenames_in_cluster])
            if cluster_idx == 0:
                distance_from_nearest_cluster = distance_from_cluster
                nearest_cluster_idx = cluster_idx
            elif distance_from_cluster < distance_from_nearest_cluster:
                distance_from_nearest_cluster = distance_from_cluster
                nearest_cluster_idx = cluster_idx
        # クラスターまでの距離が閾値を下回ればそのクラスターに所属させるがそうでなければ新たなクラスターに属する
        if distance_from_nearest_cluster < THRESHOLD:
            clusters[nearest_cluster_idx].append(filename)
        else:
            clusters.append([filename])

    # 進捗を表示する
    if (i + 1) % 1000 == 0 or i == n - 1:
        print(f'Complete {i + 1}/{n} files')


Complete 1000/30962 files
Complete 2000/30962 files
Complete 3000/30962 files
Complete 4000/30962 files
Complete 5000/30962 files
Complete 6000/30962 files
Complete 7000/30962 files
Complete 8000/30962 files
Complete 9000/30962 files
Complete 10000/30962 files
Complete 11000/30962 files
Complete 12000/30962 files
Complete 13000/30962 files
Complete 14000/30962 files
Complete 15000/30962 files
Complete 16000/30962 files
Complete 17000/30962 files
Complete 18000/30962 files
Complete 19000/30962 files
Complete 20000/30962 files
Complete 21000/30962 files
Complete 22000/30962 files
Complete 23000/30962 files
Complete 24000/30962 files
Complete 25000/30962 files
Complete 26000/30962 files
Complete 27000/30962 files
Complete 28000/30962 files
Complete 29000/30962 files
Complete 30000/30962 files


In [ ]:
len(clusters)

train/test で30962個ある画像が30887個のクラスターを形成している。

In [ ]:
cluster_sizes = [len(c) for c in clusters]
pd.Series(cluster_sizes).value_counts().sort_index().reset_index().rename(columns={'index': 'Cluster size', 0: 'Number of clusters'}).set_index('Cluster size')

2個の画像からなるクラスターが65, 3個の画像からなるクラスターが65個、5個の画像からなるクラスターが1個存在する。

## 似ている画像、重複画像の表示

複数の画像が所属するクラスターについて、同じクラスターに属する画像を横並びにして確認する。

In [ ]:
for cluster_id, image_filenames_in_cluster in tqdm(enumerate(clusters)):
    if len(image_filenames_in_cluster) < 2:
        pass
    else:
        fig = plt.figure(figsize=(18., 6.))
        for i, filename in enumerate(image_filenames_in_cluster):
            ax = plt.subplot(1, len(image_filenames_in_cluster), i + 1)
            directory = 'test' if filename in test['odai_photo_file_name'].to_list() else 'train'
            image = Image.open(os.path.join(DATA_DIR, directory, filename))
            plt.imshow(image)
            ax.set_title(f'{filename} ({directory})')
            if i == 0:
                ax.set_ylabel(f'Cluster {cluster_id}')
        plt.show()

ファイル名が異なっていても完全に、あるいはほとんど同一の画像の組み合わせが存在することを確認した。

In [ ]:
output_dir = '/content/drive/MyDrive/nishika/output'
cluster_list = []  # list of cluster_id, filename, directory
for cluster_id, image_filenames_in_cluster in tqdm(enumerate(clusters)):
    for filename in image_filenames_in_cluster:
        directory = 'test' if filename in test['odai_photo_file_name'].to_list() else 'train'
        cluster_list.append((cluster_id, filename, directory))
cluster_list = pd.DataFrame(cluster_list, columns=['cluster_id', 'filename', 'fold'])
cluster_list.to_csv(os.path.join(output_dir, 'hash_clustering_result.csv'), index=False)

0it [00:00, ?it/s]

In [ ]:
cluster_list

,cluster_id,filename,fold
0,0,9fkys1gb2r.jpg,train
1,1,c6ag0m1lak.jpg,train
2,2,whtn6gb9ww.jpg,train
3,3,6yk5cwmrsy.jpg,train
4,4,0i9gsa2jsm.jpg,train
...,...,...,...
30957,30882,0bgwr5po4l.jpg,test
30958,30883,3wgkjwrq11.jpg,test
30959,30884,rny98dohwa.jpg,test
30960,30885,rlrze2yhes.jpg,test
